<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

In [8]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Extract the table from the website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
table = soup.find('table', class_='wikitable')

#Extract the elements from the table
postcode = []
borough = []
neighborhood = []

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode.append(cells[0].find(text=True).rstrip('\n'))
        borough.append(cells[1].find(text=True).rstrip('\n'))
        neighborhood.append(cells[2].find(text=True).rstrip('\n'))
        
#Write the elements in a dataframe
df = pd.DataFrame()
df['Postcode'] = postcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

#Remove rows where Borough = Not Assigned
df = df[df["Borough"] != "Not assigned"]

#Group neighbordhoods with same postcode
for i in range(1,df.shape[0]):
    if (df.iloc[i,0] == df.iloc[i-1,0])==True:
        df.iloc[i-1,0] = 'Not assigned'
        df.iloc[i,2] = df.iloc[i-1,2] + ", " + df.iloc[i,2]
           
df = df[df["Postcode"] != "Not assigned"]

#Assign Borough name to Neighborhood if Neighborhood = Not assigned
for i in range(1,df.shape[0]):
    if (df.iloc[i,2] == "Not assigned")==True:
        df.iloc[i,2] = df.iloc[i,1]
        
df = df.reset_index(drop = True)

df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [9]:
df.shape

(103, 3)